In [3]:
import pandas as pd
import h5py
import fitsio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data_test = fitsio.read('/global/projecta/projectdirs/lsst/groups/WL/projects/wl-txpipe-hack/HSC/pdr1_wide_gama09h.fits')

In [5]:
def setup_output(path, group, cat, cols, n):
    f = h5py.File(path, 'w')
    g = f.create_group(group)
    for name in cols:
        g.create_dataset(name, shape=(n,), dtype=cat[name].dtype)
    return f

In [6]:
def write_output(output_file, group_name, cols, start, end, data):
    g = output_file[group_name]
    for name in cols:
        g[name][start:end] = data[name]

In [7]:
def shear_catalog_formatter(data_hsc, output_path, delta_gamma=0.02):
    _data = dict()
    _data['id'] = data_hsc['object_id']
    _data['ra'] = data_hsc['ira']
    _data['dec'] = data_hsc['idec']
    _data['mcal_T'] = data_hsc['ishape_hsm_moments_11']+data_hsc['ishape_hsm_moments_22']
    _data['mcal_T_1p'] = data_hsc['ishape_hsm_moments_11']+data_hsc['ishape_hsm_moments_22']
    _data['mcal_T_1m'] = data_hsc['ishape_hsm_moments_11']+data_hsc['ishape_hsm_moments_22']
    _data['mcal_T_2p'] = data_hsc['ishape_hsm_moments_11']+data_hsc['ishape_hsm_moments_22']
    _data['mcal_T_2m'] = data_hsc['ishape_hsm_moments_11']+data_hsc['ishape_hsm_moments_22']
    _data['mcal_g1'] = data_hsc['ishape_hsm_regauss_e1']
    _data['mcal_g2'] = data_hsc['ishape_hsm_regauss_e2']
    _data['mcal_s2n'] = 1./data_hsc['icmodel_mag_err']
    _data['mcal_s2n_1p'] = 1./data_hsc['icmodel_mag_err']
    _data['mcal_s2n_2p'] = 1./data_hsc['icmodel_mag_err']
    _data['mcal_s2n_1m'] = 1./data_hsc['icmodel_mag_err']
    _data['mcal_s2n_2m'] = 1./data_hsc['icmodel_mag_err']
    _data['i_mag'] = data_hsc['icmodel_mag']
    _data['i_mag_err'] = data_hsc['icmodel_mag_err']
    _data['mcal_g1_1p'] = data_hsc['ishape_hsm_regauss_e1']+delta_gamma
    _data['mcal_g2_1p'] = data_hsc['ishape_hsm_regauss_e2']+delta_gamma
    _data['mcal_g1_1m'] = data_hsc['ishape_hsm_regauss_e1']
    _data['mcal_g2_1m'] = data_hsc['ishape_hsm_regauss_e2']
    _data['mcal_g1_2p'] = data_hsc['ishape_hsm_regauss_e1']+delta_gamma
    _data['mcal_g2_2p'] = data_hsc['ishape_hsm_regauss_e2']+delta_gamma
    _data['mcal_g1_2m'] = data_hsc['ishape_hsm_regauss_e1']
    _data['mcal_g2_2m'] = data_hsc['ishape_hsm_regauss_e2']
    _data['mcal_flags'] = data_hsc['ishape_hsm_regauss_derived_shape_weight_isnull']
    Ixx = data_hsc[f'ishape_hsm_psfmoments_11']
    Iyy = data_hsc[f'ishape_hsm_psfmoments_22']
    Ixy = data_hsc[f'ishape_hsm_psfmoments_12']
    for band in ['g','r','i','z','y']:
        _data[f'mcal_mag_{band}'] = data_hsc[f'{band}cmodel_mag']
        _data[f'mcal_mag_err_{band}'] = data_hsc[f'{band}cmodel_mag_err']
        _data[f'mcal_mag_{band}_1p'] = data_hsc[f'{band}cmodel_mag']
        _data[f'mcal_mag_err_{band}_1p'] = data_hsc[f'{band}cmodel_mag_err']
        _data[f'mcal_mag_{band}_1m'] = data_hsc[f'{band}cmodel_mag']
        _data[f'mcal_mag_err_{band}_1m'] = data_hsc[f'{band}cmodel_mag_err']
        _data[f'mcal_mag_{band}_2m'] = data_hsc[f'{band}cmodel_mag']
        _data[f'mcal_mag_err_{band}_2m'] = data_hsc[f'{band}cmodel_mag_err']
        _data[f'mcal_mag_{band}_2p'] = data_hsc[f'{band}cmodel_mag']
        _data[f'mcal_mag_err_{band}_2p'] = data_hsc[f'{band}cmodel_mag_err']
    # Conversion of moments to e1, e2
    T = Ixx + Iyy
    e = (Ixx - Iyy + 2j * Ixy) / (Ixx + Iyy)
    e1 = e.real
    e2 = e.imag
    _data['mcal_psf_g1'] = e1
    _data['mcal_psf_g2'] = e2
    _data['mcal_psf_T_mean'] = T
    _data['mcal_mag'] = _data['i_mag']
    _data['mcal_mag_err'] = _data['i_mag_err']
    n = len(_data['ra'])
    _f = setup_output(output_path, 'metacal', _data, _data.keys(), n)
    write_output(_f, 'metacal', _data.keys(), 0, n, _data)
    _f.flush()
    _f.close()

In [8]:
def photometry_catalog_formatter(data_hsc, output_path):
    _data = dict()
    _data['id'] = data_hsc['object_id']
    _data['ra'] = data_hsc['ira']
    _data['dec'] = data_hsc['idec']
    for band in ['g','r','i','z','y']:
        _data[f'{band}_mag'] = data_hsc[f'{band}cmodel_mag']
        _data[f'{band}_mag_err'] = data_hsc[f'{band}cmodel_mag_err']
        _data[f'snr_{band}'] = 1./data_hsc[f'{band}cmodel_mag_err']
    _data['u_mag']=99*np.ones(len(_data['ra']))
    _data['u_mag_err']=99*np.ones(len(_data['ra']))
    #_catalog_out = h5py.File(output_path, 'w')
    #_catalog_out.create_dataset('photometry', data=pd.DataFrame(_data).to_records())
    #_catalog_out.flush()
    n = len(_data['ra'])
    _f = setup_output(output_path, 'photometry', _data, _data.keys(), n)
    write_output(_f, 'photometry', _data.keys(), 0, n, _data)
    _f.flush()
    _f.close()

In [29]:
def star_catalog_formatter(data_hsc, output_path):
    star_data = {}
    # We specifically use the stars chosen for PSF measurement
    #star = data_hsc['icalib_psf_used']
    star = data_hsc['iclassification_extendedness'] == 0
    # General columns
    star_data['ra'] = data_hsc['ira'][star]
    star_data['dec'] = data_hsc['idec'][star]
    star_data['id'] = data_hsc['object_id'][star]
    for band in 'grizy':
        star_data[f'{band}_mag'] = data_hsc[f'{band}cmodel_mag'][star]
    star_data['u_mag'] = 99*np.ones(len(star_data['id']))
    star_data['u_mag_err'] = 99*np.ones(len(star_data['id']))
    kinds = [
            ('', 'measured_'),
            ('psf', 'model_')
        ]

    for in_name, out_name in kinds:
        # Pulling out the correct moment columns
        Ixx = data_hsc[f'ishape_hsm_{in_name}moments_11'][star]
        Iyy = data_hsc[f'ishape_hsm_{in_name}moments_22'][star]
        Ixy = data_hsc[f'ishape_hsm_{in_name}moments_12'][star]

        # Conversion of moments to e1, e2
        T = Ixx + Iyy
        e = (Ixx - Iyy + 2j * Ixy) / (Ixx + Iyy)
        e1 = e.real
        e2 = e.imag

        # save to output
        star_data[f'{out_name}e1'] = e1
        star_data[f'{out_name}e2'] = e2
        star_data[f'{out_name}T'] = T
    #_catalog_out = h5py.File(output_path, 'a')
    #_catalog_out.create_dataset('star_catalog', data=pd.DataFrame(star_data).to_records())
    #_catalog_out.flush()
    n = len(star_data['ra'])
    _f = setup_output(output_path, 'stars', star_data, star_data.keys(), n)
    write_output(_f, 'stars', star_data.keys(), 0, n, star_data)
    _f.flush()
    _f.close()

In [32]:
star_catalog_formatter(data_test, '/global/cscratch1/sd/jsanch87/HSC_pdr1_wide_gama09h_star_catalog.hdf5')

In [20]:
shear_catalog_formatter(data_test, '/global/cscratch1/sd/jsanch87/HSC_pdr1_wide_gama09h_shear.hdf5')

In [38]:
photometry_catalog_formatter(data_test, '/global/cscratch1/sd/jsanch87/HSC_pdr1_wide_gama09h_photo.hdf5')